In [32]:
import torch
from torch import nn
import matplotlib.pyplot as plt

print(torch.__version__)

2.5.1


In [33]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [34]:
from sklearn.datasets import make_circles
n_samples = 1000
X, y = make_circles(n_samples, noise=0.03, random_state=42)
X[:5], y[:5]

(array([[ 0.75424625,  0.23148074],
        [-0.75615888,  0.15325888],
        [-0.81539193,  0.17328203],
        [-0.39373073,  0.69288277],
        [ 0.44220765, -0.89672343]]),
 array([1, 1, 1, 1, 0]))

In [35]:
import pandas as pd
circles = pd.DataFrame({"X1": X[:, 0], "X2": X[:, 1], "y": y})
circles.head()

,X1,X2,y
0,0.754246,0.231481,1
1,-0.756159,0.153259,1
2,-0.815392,0.173282,1
3,-0.393731,0.692883,1
4,0.442208,-0.896723,0


In [36]:
circles.y.value_counts()

y
1    500
0    500
Name: count, dtype: int64

In [37]:
from sklearn.model_selection import train_test_split
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
X = torch.from_numpy(X).type(torch.float).to(device)
y = torch.from_numpy(y).type(torch.float).to(device)

X[:5], y[:5]

(tensor([[ 0.7542,  0.2315],
         [-0.7562,  0.1533],
         [-0.8154,  0.1733],
         [-0.3937,  0.6929],
         [ 0.4422, -0.8967]], device='mps:0'),
 tensor([1., 1., 1., 1., 0.], device='mps:0'))

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, 

(torch.Size([800, 2]),
 torch.Size([800]),
 torch.Size([200, 2]),
 torch.Size([200]))

In [41]:
class CircleModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(in_features=2, out_features=5)
        self.linear_2 = nn.Linear(in_features=5, out_features=1)

    def forward(self, X):
        return self.linear_2(self.linear_1(X))

model_v0 = CircleModelV0()
model_v0.to(device)
print(f' Model on device: {next(model_v0.parameters()).device}')

loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model_v0.parameters(), lr=0.01)

epochs = 100

for epoch in range(epochs):
    model_v0.train()
    y_train_pred = model_v0(X_train)
    train_loss = loss_fn(y_train_pred, y_train)
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # test
    model_v0.eval()
    with torch.inference_mode():
        y_test_pred = model_v0(X_test)
    test_loss = loss_fn(y_test_pred, y_test)

 Model on device: mps:0


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([800])) that is different to the input size (torch.Size([800, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
